# Streaming

Streaming is an important UX consideration for LLM apps, and agents are no exception. Streaming with agents is made more complicated by the fact that it's not just tokens that you will want to stream, but you may also want to stream back the intermediate steps an agent takes.

Let's take a look at how to do this.

## Set up the agent

Let's set up a simple agent for demonstration purposes. For our tool, we will use [Tavily](/docs/integrations/tools/tavily_search). Make sure that you've exported an API key with 

```bash
export TAVILY_API_KEY="..."
```

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
tools = [search]

We will use a prompt from the hub - you can inspect the prompt more at [https://smith.langchain.com/hub/hwchase17/openai-functions-agent](https://smith.langchain.com/hub/hwchase17/openai-functions-agent)

In [3]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.chat_models import ChatOpenAI

# Get the prompt to use - you can modify this!
# If you want to see the prompt in full, you can at: https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

## Stream intermediate steps

Let's look at how to stream intermediate steps. We can do this easily by just using the `.stream` method on the AgentExecutor

In [4]:
for chunk in agent_executor.stream({"input": "what is the weather in SF and then LA"}):
    print(chunk)
    print("------")

{'actions': [AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})])], 'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})]}
------
{'steps': [AgentStep(action=AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})]), observation=[{'

You can see that we get back a bunch of different information. There are two ways to work with this information:

1. By using the AgentAction/observation/AgentFinish object
2. By using the `messages` object

You may prefer to use the `messages` object if you are working with a chatbot - because these are chat messages and can be rendered directly. If you don't care about that, the AgentAction/observation/AgentFinish is probably the easier one to inspect.

### Using AgentAction/observation/AgentFinish

You can access these raw objects as part of the streamed payload. This gives you more low level information, but can be harder to parse.

In [6]:
for chunk in agent_executor.stream({"input": "what is the weather in SF and then LA"}):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(
                f"Calling Tool ```{action.tool}``` with input ```{action.tool_input}```"
            )
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Got result: ```{step.observation}```")
    # Final result
    elif "output" in chunk:
        print(chunk["output"])
    else:
        raise ValueError
    print("------")

Calling Tool ```tavily_search_results_json``` with input ```{'query': 'weather in San Francisco'}```
------
Got result: ```[{'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US', 'content': 'recents Specialty Forecasts 10 Day Weather-San Francisco, CA Today Mon 18 | Day  Fri 22 Fri 22 | Day Foggy early, then partly cloudy later in the day. High around 60F. Winds W at 10 to 15 mph.  Considerable cloudiness with occasional rain showers. High 59F. Winds SSE at 5 to 10 mph. Chance of rain 50%.  Thu 28 | Night Cloudy with showers. Low 46F. Winds S at 5 to 10 mph. Chance of rain 40%. Fri 29 Fri 29 | Day10 Day Weather-San Francisco, CA As of 5:52 pm PST alertLevel3 Coastal Flood Advisory+1 More Tonight Mostly Cloudy Night --/44° Rain 7% Arrow Up Sun 24| Night 44° Mostly Cloudy Night Rain 7%...'}]```
------
Calling Tool ```tavily_search_results_json``` with input ```{'query': 'weather in Los Angeles'}```
------
Got result: ```[{'url': 'https://hoodline.com/2023/12/los-a

### Using messages

Using messages can be nice when working with chat applications - because everything is a message!

In [7]:
for chunk in agent_executor.stream({"input": "what is the weather in SF and then LA"}):
    print(chunk["messages"])
    print("------")

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})]
------
[FunctionMessage(content='[{"url": "https://www.cbsnews.com/sanfrancisco/news/next-round-of-rain-set-to-arrive-in-bay-area-wednesday-morning/", "content": "weather persists through Thursday morning. The second system is projected to get to the Bay Area early Friday,  Watch CBS News Next round of rain set to arrive in Bay Area Wednesday morning December 26, 2023 / 8:17 AM PST  to the Bay Area on Wednesday with the arrival of the first of two storm systems.  Overnight lows should be mostly in the 40s in the region, with some areas around the bay dropping into the 50s.Watch CBS News Weather Next round of rain set to arrive in Bay Area Wednesday morning December 26, 2023 / 8:17 AM PST / CBS/Bay City News Service While the outlook on Tuesday is cloudy and..."}, {"url": "https://weather.com/weather/tenday/l/San Francisco CA US

## Stream tokens

In addition to streaming the final result, you can also stream tokens. This will require slightly more complicated parsing of the logs

You will also need to make sure you set the LLM to be streaming

In [8]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [9]:
async for chunk in agent_executor.astream_log(
    {"input": "what is the weather in sf", "chat_history": []},
    include_names=["ChatOpenAI"],
):
    print(chunk)

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '32650ba8-8a53-4b76-8846-dbb6c3a65727',
            'logs': {},
            'streamed_output': []}})
RunLogPatch({'op': 'add',
  'path': '/logs/ChatOpenAI',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'ce3a507d-210d-40aa-8576-dd0aa97e6498',
            'metadata': {},
            'name': 'ChatOpenAI',
            'start_time': '2023-12-26T17:55:56.653',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:3'],
            'type': 'llm'}})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatOpenAI/streamed_output/-',
  'value': AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '', 'name': 'tavily_search_results_json'}})})
RunLogPatch({'op': 'add', 'path': '/logs/ChatOpenAI/streamed_output_str/-', 'value':

This may require some logic to get in a workable format

In [10]:
path_status = {}
async for chunk in agent_executor.astream_log(
    {"input": "what is the weather in sf", "chat_history": []},
    include_names=["ChatOpenAI"],
):
    for op in chunk.ops:
        if op["op"] == "add":
            if op["path"] not in path_status:
                path_status[op["path"]] = op["value"]
            else:
                path_status[op["path"]] += op["value"]
    print(op["path"])
    print(path_status.get(op["path"]))
    print("----")


None
----
/logs/ChatOpenAI
{'id': '3f6d3587-600f-419b-8225-8908a347b7d2', 'name': 'ChatOpenAI', 'type': 'llm', 'tags': ['seq:step:3'], 'metadata': {}, 'start_time': '2023-12-26T17:56:19.884', 'streamed_output': [], 'streamed_output_str': [], 'final_output': None, 'end_time': None}
----
/logs/ChatOpenAI/streamed_output/-
content='' additional_kwargs={'function_call': {'arguments': '', 'name': 'tavily_search_results_json'}}
----
/logs/ChatOpenAI/streamed_output/-
content='' additional_kwargs={'function_call': {'arguments': '{\n', 'name': 'tavily_search_results_json'}}
----
/logs/ChatOpenAI/streamed_output/-
content='' additional_kwargs={'function_call': {'arguments': '{\n ', 'name': 'tavily_search_results_json'}}
----
/logs/ChatOpenAI/streamed_output/-
content='' additional_kwargs={'function_call': {'arguments': '{\n  "', 'name': 'tavily_search_results_json'}}
----
/logs/ChatOpenAI/streamed_output/-
content='' additional_kwargs={'function_call': {'arguments': '{\n  "query', 'name': 'tav